In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import matplotlib as plt
import torch
from torch.autograd import Variable

In [2]:
fil="C:/Users/Zenodia/Rscripts/data/"
df=pd.read_csv(fil+"timeseries_example.csv", sep=";")
df.head()

,Month of Year,Sessions
0,200901,66953
1,200902,61591
2,200903,87241
3,200904,85263
4,200905,100813


In [3]:
df['Month of Year']=df['Month of Year'].astype("str")
df['month']=df['Month of Year'].apply(lambda x: x[4:])
df['year']=df['Month of Year'].apply(lambda x: x[:4])
df.head()

,Month of Year,Sessions,month,year
0,200901,66953,01,2009
1,200902,61591,02,2009
2,200903,87241,03,2009
3,200904,85263,04,2009
4,200905,100813,05,2009


In [4]:
cuts=[df['Sessions'].quantile(.1*i) for i in range(0,10)]
print (cuts , df['Sessions'].quantile(.90))



[52735.0, 69856.2, 77730.8, 87020.1, 94290.20000000001, 100113.0, 108201.00000000003, 124658.0, 136022.2, 151519.7] 151519.7


In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
# convert Session data to symbols from a,b,c,d,e,f,g,h,i,j
symbols=[]
for idx in df.index:
    if df.iloc[idx,1]< cuts[0]:
        symbols.append(normalizeString('a'))
    elif (df.iloc[idx,1]>=cuts[0] and df.iloc[idx,1] < cuts[1]):
        symbols.append(normalizeString('b'))
    elif (df.iloc[idx,1]>=cuts[1] and df.iloc[idx,1] < cuts[2]):
        symbols.append(normalizeString('c'))
    elif (df.iloc[idx,1]>=cuts[2] and df.iloc[idx,1] < cuts[3]):
        symbols.append(normalizeString('d'))
    elif (df.iloc[idx,1]>=cuts[3] and df.iloc[idx,1] < cuts[4]):
        symbols.append(normalizeString('e'))
    elif (df.iloc[idx,1]>=cuts[4] and df.iloc[idx,1] < cuts[5]):
        symbols.append(normalizeString('f'))
    elif (df.iloc[idx,1]>=cuts[5] and df.iloc[idx,1] < cuts[6]):
        symbols.append(normalizeString('g'))
    elif (df.iloc[idx,1]>=cuts[6] and df.iloc[idx,1] < cuts[7]):
        symbols.append(normalizeString('h'))
    elif (df.iloc[idx,1]>=cuts[7] and df.iloc[idx,1] < cuts[8]):
        symbols.append(normalizeString('i'))
    elif (df.iloc[idx,1]>=cuts[8] and df.iloc[idx,1] < cuts[9]):
        symbols.append(normalizeString('j'))
    else :
        symbols.append(normalizeString('k'))
print(len(symbols),symbols)

84 ['b', 'b', 'e', 'd', 'g', 'g', 'h', 'f', 'c', 'b', 'b', 'b', 'c', 'c', 'f', 'k', 'j', 'g', 'h', 'h', 'd', 'c', 'b', 'd', 'e', 'e', 'f', 'f', 'i', 'i', 'i', 'j', 'e', 'd', 'c', 'b', 'g', 'e', 'g', 'h', 'h', 'i', 'j', 'j', 'e', 'd', 'b', 'b', 'h', 'g', 'i', 'h', 'i', 'j', 'k', 'k', 'e', 'f', 'c', 'e', 'g', 'd', 'f', 'i', 'i', 'k', 'k', 'j', 'd', 'e', 'c', 'c', 'j', 'h', 'k', 'h', 'j', 'k', 'k', 'k', 'g', 'f', 'd', 'f']


In [7]:
df['symbols']=symbols
print ( df.year.unique())
df.head()

['2009' '2010' '2011' '2012' '2013' '2014' '2015']


,Month of Year,Sessions,month,year,symbols
0,200901,66953,01,2009,b
1,200902,61591,02,2009,b
2,200903,87241,03,2009,e
3,200904,85263,04,2009,d
4,200905,100813,05,2009,g


In [8]:
lookup_symbols=dict(zip(df['Sessions'],df['symbols']))
lookup_symbols.items()

dict_items([(87042, 'e'), (100100, 'f'), (84997, 'd'), (66953, 'b'), (160778, 'k'), (61591, 'b'), (98061, 'f'), (85263, 'd'), (94170, 'e'), (184984, 'k'), (76179, 'c'), (120469, 'h'), (151190, 'j'), (63127, 'b'), (134574, 'i'), (98969, 'f'), (67868, 'b'), (105117, 'g'), (100126, 'g'), (151173, 'j'), (145056, 'j'), (183856, 'k'), (88866, 'e'), (151661, 'k'), (79689, 'd'), (115623, 'h'), (137000, 'j'), (127260, 'i'), (124586, 'h'), (101959, 'g'), (100268, 'g'), (75694, 'c'), (77872, 'd'), (86823, 'd'), (94771, 'f'), (130357, 'i'), (100534, 'g'), (87305, 'e'), (136159, 'j'), (96439, 'f'), (140222, 'j'), (71488, 'c'), (147198, 'j'), (70083, 'c'), (134340, 'i'), (132918, 'i'), (93639, 'e'), (111176, 'h'), (87241, 'e'), (92874, 'e'), (100407, 'g'), (100813, 'g'), (152014, 'k'), (63757, 'b'), (108972, 'h'), (115923, 'h'), (78804, 'd'), (92757, 'e'), (52735, 'b'), (120154, 'h'), (77519, 'c'), (95226, 'f'), (76383, 'c'), (178785, 'k'), (68835, 'b'), (167905, 'k'), (154854, 'k'), (86293, 'd'), (

In [9]:
train_data=[]

yrs=df['year'].unique().tolist()
for yr in yrs:
    
    temp=[lookup_symbols[x] for x in df.loc[(df.year==str(yr))]['Sessions'].values.tolist()]
    train_data.append(temp)

print(train_data,len(train_data))

[['b', 'b', 'e', 'd', 'g', 'g', 'h', 'f', 'c', 'b', 'b', 'b'], ['c', 'c', 'f', 'k', 'j', 'g', 'h', 'h', 'd', 'c', 'b', 'd'], ['e', 'e', 'f', 'f', 'i', 'i', 'i', 'j', 'e', 'd', 'c', 'b'], ['g', 'e', 'g', 'h', 'h', 'i', 'j', 'j', 'e', 'd', 'b', 'b'], ['h', 'g', 'i', 'h', 'i', 'j', 'k', 'k', 'e', 'f', 'c', 'e'], ['g', 'd', 'f', 'i', 'i', 'k', 'k', 'j', 'd', 'e', 'c', 'c'], ['j', 'h', 'k', 'h', 'j', 'k', 'k', 'k', 'g', 'f', 'd', 'f']] 7


In [10]:
word_to_ix = {}
for sent in train_data:
    #print("sent:{}, tags:{}".format(sent,tags))
    for word in sent:
        #print("word:{}".format(word))
        #print("word_to_idx", word_to_ix)
        if str(word) not in word_to_ix:
            word_to_ix[str(word)] = len(word_to_ix)
print(word_to_ix)

{'f': 5, 'g': 3, 'j': 8, 'k': 7, 'i': 9, 'h': 4, 'b': 0, 'c': 6, 'e': 1, 'd': 2}


In [11]:
target=[]
for item in train_data:
    if 'k' in item:
        target.append([1])
    else:
        target.append([0])
print(target)
print(target[-1][0])

[[0], [1], [0], [0], [1], [1], [1]]
1


In [12]:
# RNN Model (Many-to-One)
# Hyper Parameters
sequence_length = 12
input_size = 12
hidden_size = 128
num_layers = 2
num_classes = 2
batch_size = 1
num_epochs = 100
learning_rate = 0.01
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        #print("forward x",x.size())
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        #print("forward out1",out)
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        #print("forward out2",out)
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)

In [13]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    
# Train the Model
for epoch in range(num_epochs):

    for i in range(len(train_data)-1):
        optimizer.zero_grad()
        tryin=[word_to_ix[x] for x in list(train_data[i])] 
        input_seq= Variable(torch.FloatTensor([tryin])).unsqueeze(0)
        output_seq = rnn(input_seq)
        last_output = output_seq

        err = criterion(last_output, Variable(torch.LongTensor(target[i])))
        err.backward()
        optimizer.step()
        if epoch%10 ==0 :
            print("loss",err)

loss Variable containing:
 0.6960
[torch.FloatTensor of size 1]

loss Variable containing:
 0.9105
[torch.FloatTensor of size 1]

loss Variable containing:
 0.6081
[torch.FloatTensor of size 1]

loss Variable containing:
 0.6017
[torch.FloatTensor of size 1]

loss Variable containing:
 0.8530
[torch.FloatTensor of size 1]

loss Variable containing:
 0.7926
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-09 *
  2.6659
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-10 *
  5.2066
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-06 *
  1.9760
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-09 *
  4.6892
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-10 *
  2.7347
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-10 *
  2.6272
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-09 *
  1.7782
[torch.FloatTensor of size 1]

loss Variable containing:
1.00000e-10 *
  3.9459
[

In [14]:
# Test the Model
tryin=[word_to_ix[x] for x in list(train_data[-1])] 
input_seq= Variable(torch.FloatTensor([tryin])).unsqueeze(0)
output_seq = rnn(input_seq)
_, pred = torch.max(output_seq.data, 1)
print("pred",pred)
print(type(pred[0,0]))

if pred[0,0]==target[-1][0]:
    print("correct")
else:
    print("incorrect")


pred 
 1
[torch.LongTensor of size 1x1]

<class 'int'>
correct
